In [19]:
#from config import (census_key, gkey)
import numpy as np
import pandas as pd
import requests
import zillow
import pprint as pp

In [20]:
#----API Reference GitHub --> https://github.com/seme0021/python-zillow

api = zillow.ValuationApi()
key = "X1-ZWz17fdl35adqj_3bebs"

In [21]:
#address = input('123 Streetname Rd, City, State ')
#postal_code = input('Zip Code')

In [22]:
#TEST ADDRESS: 14 Hayes Ave, Elizabeth, NJ 07202

address = "501 Shumaker Dr, Monroeville, PA"
postal_code = "15146"


request_data = api.GetSearchResults(key, address, postal_code)

In [29]:
result = request_data.get_dict()
result

{'zpid': '11626766',
 'similarity_score': None,
 'links': {'home_details': 'https://www.zillow.com/homedetails/501-Shumaker-Dr-Monroeville-PA-15146/11626766_zpid/',
  'graphs_and_data': 'http://www.zillow.com/homedetails/501-Shumaker-Dr-Monroeville-PA-15146/11626766_zpid/#charts-and-data',
  'map_this_home': 'http://www.zillow.com/homes/11626766_zpid/',
  'comparables': 'http://www.zillow.com/homes/comps/11626766_zpid/'},
 'full_address': {'street': '501 Shumaker Dr',
  'zipcode': '15146',
  'city': 'Monroeville',
  'state': 'PA',
  'latitude': '40.456694',
  'longitude': '-79.763917'},
 'zestimate': {'amount': 191457,
  'amount_currency': 'USD',
  'amount_last_updated': '02/28/2020',
  'amount_change_30days': 2696,
  'valuation_range_low': 181884,
  'valuation_range_high': 214432},
 'local_realestate': {'region_name': 'Monroeville',
  'region_id': '46582',
  'region_type': 'city',
  'overview_link': 'http://www.zillow.com/local-info/PA-Monroeville/r_46582/',
  'fsbo_link': 'http://www

In [13]:
zestimate = result['zestimate']
zestimate

{'amount': 191457,
 'amount_currency': 'USD',
 'amount_last_updated': '02/28/2020',
 'amount_change_30days': 2696,
 'valuation_range_low': 181884,
 'valuation_range_high': 214432}

In [36]:
address = result['full_address']
address

{'street': '501 Shumaker Dr',
 'zipcode': '15146',
 'city': 'Monroeville',
 'state': 'PA',
 'latitude': '40.456694',
 'longitude': '-79.763917'}

In [38]:
address['street']


'501 Shumaker Dr'

In [ ]:
address = "501 Shumaker Dr, Monroeville, PA"
postal_code = "15146"

def address_search(address, postal_code):
    zestimate_list = []
    request_data = api.GetSearchResults(key, address, postal_code)
    result = request_data.get_dict()
    
    zestimate_val = result['zestimate']
    print(zestimate_val['amount'])
    
    
    


In [45]:
address_search("501 Shumaker Dr, Monroeville, PA","15146")

191457
